In [1]:
import torch
import torch.nn as nn


class SimpleAttentionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleAttentionModel, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.self_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=4)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Flatten the input for MNIST
        x = x.view(x.size(0), -1)  # (batch_size, input_dim)
        x = self.input_layer(x).unsqueeze(1)  # Add sequence length dimension -> (batch_size, seq_len, hidden_dim)

        # Self-Attention
        x, _ = self.self_attention(x, x, x)  # (batch_size, seq_len, hidden_dim)

        # Take the first token as a representation (like CLS token)
        x = x.squeeze(1)
        x = self.dropout(x)

        # Output
        x = self.output_layer(x)
        return x


In [2]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Hyperparameters
input_dim = 28 * 28
hidden_dim = 128
output_dim = 10
batch_size = 64
learning_rate = 0.001
epochs = 10

# Data Loaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model, Loss, Optimizer
model = SimpleAttentionModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        total += target.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 0.6194
Epoch 2/10, Loss: 0.4314
Epoch 3/10, Loss: 0.4333
Epoch 4/10, Loss: 0.5001
Epoch 5/10, Loss: 0.6412
Epoch 6/10, Loss: 0.7154
Epoch 7/10, Loss: 0.8006
Epoch 8/10, Loss: 0.9271
Epoch 9/10, Loss: 1.1371
Epoch 10/10, Loss: 1.0617
Test Accuracy: 0.6174
